In [1]:
import numpy as np
import pandas as pd
import wrangle_products as w
import re
import nltk

In [2]:
products = w.wrangle_products()

Opening local amazon_cycling.csv file


In [3]:
products.head(2)

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating
0,QUXIS Portable High Pressure Foot Air Pump Hea...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/QUXIS-Portable-Activated...,4.0,7635,5.99,15.59,quxis portable high pressure foot air pump hea...,9.6,30540.0
1,Boldfit Gym Shaker for Protein Shake Leakproof...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Boldfit-Typhoon-Shaker-L...,4.1,8052,2.75,2.75,boldfit gym shaker protein shake leakproof sha...,0.0,33013.2


In [4]:
products.tail()

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating
1112,Cread Craft Winter Wear Men's Poly Cotton Ther...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/61t6DNxN3+...,https://www.amazon.in/Winter-Cotton-Thermal-Se...,0.0,0,5.06,17.99,cread craft winter wear men poly cotton therma...,12.93,0.0
1113,Lux Inferno Women's Round Neck Sleeveless Uppe...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/41KdM4GjzZ...,https://www.amazon.in/Lux-Inferno-Womens-Sleev...,0.0,0,5.68,5.68,lux inferno woman round neck sleeveless upper ...,0.00,0.0
1114,"WEGAS Bigbull 26 Inch Mountain Bike 18.5"" Fram...",sports & fitness,Cycling,https://m.media-amazon.com/images/I/81AoVzv-8s...,https://www.amazon.in/WEGAS-26-Mountain-Standa...,4.3,2,171.00,179.99,wegas bigbull 26 inch mountain bike 185 frame ...,8.99,8.6
1115,AN Collection Socks Winter Warmer Thicken Ther...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/31u52aecz3...,https://www.amazon.in/Collection-Winter-Thicke...,0.0,0,3.23,3.59,collection sock winter warmer thicken thermal ...,0.36,0.0
1116,Unisex Stretchable Woollen Sleeveless Thermal ...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/71UXyugLMJ...,https://www.amazon.in/Unisex-STREATCHABLE-Wool...,0.0,0,5.39,7.19,unisex stretchable woollen sleeveless thermal ...,1.80,0.0


In [5]:
products.shape

(1117, 12)

In [6]:
sub_cat_list = ['Tires and Tubes', 'Gloves', 'Helmets', 'Bike Stands',
       'Mountain Bikes', 'Hydration Packs', 'Jerseys', 'Fenders',
       'Cleaners', 'Socks', 'Caps', 'Touring Bikes', 'Bottles and Cages',
       'Vests', 'Road Bikes', 'Bike Racks', 'Shorts']

In [7]:
cat_list = w.get_cat_list(sub_cat_list, extra_words=['bike', 'bikes'])

### To find the product categories: we will loop through the product list, then on each product, we can loop through the cat_list, and check for matches using regex. If we find a match, we will return the product row index number, and category number based on position in the cat_list. We should also account for products having no matches or many category matches

#### lets start by figuring out the regex match formula for one cat and one row

#### regexp = r'.*{cat}.*'
regex doesn't seem to be able to handle dynamic f string style commands so lets build a table with the original category name, the lemmatized version. along with the regexp to find the lemmatized cat name

In [8]:
cat_df = pd.concat([
    pd.Series(sub_cat_list), 
    pd.Series(cat_list)
    ], axis=1).\
    rename(columns={0:'shop_cat', 1:'lemmed_cat'})

In [9]:
cat_df

,shop_cat,lemmed_cat
0,Tires and Tubes,tire tube
1,Gloves,glove
2,Helmets,helmet
3,Bike Stands,stand
4,Mountain Bikes,mountain
5,Hydration Packs,hydration pack
6,Jerseys,jersey
7,Fenders,fender
8,Cleaners,cleaner
9,Socks,sock


In [10]:
regexp = r'.*tire.*|.*tube.*'

In [11]:
regexp = r'.*horn.*|.*portable.*'

In [12]:
re.search(regexp, products.name_preped[0])

<re.Match object; span=(0, 108), match='quxis portable high pressure foot air pump heavy >

### Lets make a list of regular expressions for each category

In [13]:
regex_list = [r'.*tire.*|.*tube.*',
              r'.*glove.*',
              r'.*helmet.*',
              r'.*stand.*',
              r'.*mountain.*',
              r'.*hydration.*',
              r'.*jersey.*',
              r'.*fender.*',
              r'.*cleaner.*',
              r'.*sock.*',
              r'.*cap.*',
              r'.*touring.*',
              r'.*bottle.*|.*cage.*',
              r'.*vest.*',
              r'.*road.*',
              r'.*rack.*',
              r'.*short.*']

In [14]:
cat_df = pd.concat([cat_df, pd.Series(regex_list)], axis=1).rename(columns={0:'regexp'})

In [15]:
cat_df

,shop_cat,lemmed_cat,regexp
0,Tires and Tubes,tire tube,.*tire.*|.*tube.*
1,Gloves,glove,.*glove.*
2,Helmets,helmet,.*helmet.*
3,Bike Stands,stand,.*stand.*
4,Mountain Bikes,mountain,.*mountain.*
5,Hydration Packs,hydration pack,.*hydration.*
6,Jerseys,jersey,.*jersey.*
7,Fenders,fender,.*fender.*
8,Cleaners,cleaner,.*cleaner.*
9,Socks,sock,.*sock.*


### Time to loop. We will loop through the products.name_preped, and on each product, loop through cat_df.regexp to find matches. We will then create a list of products for each category. So we will want the product row# along with the category# for any matches.

In [16]:
for i, product in enumerate(products.name_preped[7:8]):
    print(i)
    print(product)
    for j, cat in enumerate(cat_df.regexp):
        print(j)
        print(cat)
        if re.search(cat, product):
            print('yes')

0
pursue fashion premium cotton ankle sock men woman free size solid unisex ankle length sock pack 5 multi
0
.*tire.*|.*tube.*
1
.*glove.*
2
.*helmet.*
3
.*stand.*
4
.*mountain.*
5
.*hydration.*
6
.*jersey.*
7
.*fender.*
8
.*cleaner.*
9
.*sock.*
yes
10
.*cap.*
11
.*touring.*
12
.*bottle.*|.*cage.*
13
.*vest.*
14
.*road.*
15
.*rack.*
16
.*short.*


In [17]:
cat_matches = {}
for i, product in enumerate(products.name_preped):
    matching_cats = []
#     print(i)
#     print(product)
    for j, cat in enumerate(cat_df.regexp):
#         print(j)
#         print(cat)
        if re.search(cat, product):
            matching_cats.append(cat_df.shop_cat[j])
#         for k in matching_cats:
        cat_matches[i] = matching_cats
cat_matches

{0: [],
 1: ['Bottles and Cages'],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: ['Socks'],
 8: [],
 9: [],
 10: [],
 11: [],
 12: ['Bottles and Cages'],
 13: ['Mountain Bikes'],
 14: [],
 15: [],
 16: [],
 17: ['Bottles and Cages'],
 18: ['Bottles and Cages'],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: ['Mountain Bikes'],
 26: ['Road Bikes'],
 27: ['Bottles and Cages'],
 28: ['Socks'],
 29: [],
 30: [],
 31: [],
 32: ['Socks'],
 33: [],
 34: [],
 35: [],
 36: [],
 37: ['Shorts'],
 38: ['Helmets', 'Mountain Bikes'],
 39: ['Mountain Bikes'],
 40: [],
 41: [],
 42: ['Mountain Bikes'],
 43: [],
 44: [],
 45: ['Socks'],
 46: [],
 47: ['Bike Stands', 'Mountain Bikes', 'Road Bikes'],
 48: ['Mountain Bikes'],
 49: ['Socks'],
 50: ['Hydration Packs'],
 51: [],
 52: ['Socks'],
 53: ['Bottles and Cages'],
 54: [],
 55: [],
 56: [],
 57: ['Bottles and Cages'],
 58: [],
 59: [],
 60: ['Socks'],
 61: ['Bottles and Cages'],
 62: [],
 63: [],
 64: [],
 65: ['Socks'],
 66: ['Socks'],
 67

In [18]:
cat_matches[998][0]

'Socks'

In [19]:
len(cat_matches)

1117

In [20]:
products = pd.concat([products, pd.Series(cat_matches)], axis=1).\
        rename(columns={0:'cat_matches'})

In [21]:
products.head()

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating,cat_matches
0,QUXIS Portable High Pressure Foot Air Pump Hea...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/QUXIS-Portable-Activated...,4.0,7635,5.99,15.59,quxis portable high pressure foot air pump hea...,9.60,30540.0,[]
1,Boldfit Gym Shaker for Protein Shake Leakproof...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Boldfit-Typhoon-Shaker-L...,4.1,8052,2.75,2.75,boldfit gym shaker protein shake leakproof sha...,0.00,33013.2,[Bottles and Cages]
2,Techista 2-in-1 Rechargeable - Cycle Light (3 ...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Techista-2-1-Rechargeabl...,4.1,2214,4.07,11.99,techista 2in1 rechargeable cycle light 3 mode ...,7.92,9077.4,[]
3,"Nivia Plain Encounter Stockings (L, Black) - P...",sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Nivia-728LBW-Encounter-B...,4.1,2974,1.74,1.79,nivia plain encounter stocking l black polyest...,0.05,12193.4,[]
4,Eazo Steel Multipurpose Air Pump with needle (...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/EAZO-Steel-Multipurpose-...,4.1,3678,4.43,5.40,eazo steel multipurpose air pump needle use ca...,0.97,15079.8,[]


## Switching gears. We are going to look at recommending products based on avg cluster spending amount, and product review scores, then work on getting category matches later

In [22]:
products.describe()

,ratings,no_of_ratings,discount_price,actual_price,discount_amount,prod_rating
count,1117.000000,1117.000000,1117.000000,1117.000000,1117.000000,1117.000000
mean,2.774575,227.228290,19.441128,31.711889,12.270761,944.593912
std,1.798072,1322.349807,44.937416,69.498948,37.147052,5690.028019
min,0.000000,0.000000,0.710000,1.260000,0.000000,0.000000
25%,0.000000,0.000000,3.660000,7.190000,1.760000,0.000000
50%,3.600000,4.000000,5.990000,11.990000,4.800000,17.200000
75%,4.000000,29.000000,11.770000,19.190000,8.990000,108.800000
max,5.000000,19181.000000,695.880000,719.990000,558.600000,86314.500000


#### Over 75% of the product dataset is priced less than $20

- cluster 0 average revenue is: 179.51208083174697
- cluster 1 average revenue is: 321.6032662487628
- cluster 2 average revenue is: 898.5687723480333
- cluster 3 average revenue is: 2276.8223954060704

### Lets see if we can group products into
- less than 20 dollars
- 20 to 40 dollars
- 40 to 100 dollars
- above 100 dollars

In [23]:
len(products[(products.actual_price <20)])

848

In [24]:
len(products[(products.actual_price >= 20) & (products.actual_price < 40)])

111

In [25]:
len(products[(products.actual_price >= 40) & (products.actual_price < 100)])

69

In [26]:
len(products[(products.actual_price >= 100)])

89

In [27]:
group_0 = products[(products.actual_price <20)]
group_1 = products[(products.actual_price >= 20) & (products.actual_price < 40)]
group_2 = products[(products.actual_price >= 40) & (products.actual_price < 100)]
group_3 = products[(products.actual_price >= 100)]

In [28]:
group_0 = group_0.sort_values('prod_rating', ascending=False).\
        reset_index().drop(columns='index')
group_1 = group_1.sort_values('prod_rating', ascending=False).\
        reset_index().drop(columns='index')
group_2 = group_2.sort_values('prod_rating', ascending=False).\
        reset_index().drop(columns='index')
group_3 = group_3.sort_values('prod_rating', ascending=False).\
        reset_index().drop(columns='index')

In [29]:
group_3.head(10)

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating,cat_matches
0,Bruder Healthcare Co Medi Beads Eye Hydrating ...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/71gUQvt2ff...,https://www.amazon.in/Medi-Beads-Eye-Hydrating...,4.5,19181,57.95,148.80,bruder healthcare co medi bead eye hydrating c...,90.85,86314.5,[]
1,"Fitbit Ace 2 Activity Tracker for Kids, One Size",sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Fitbit-Activity-Tracker-...,4.6,16596,284.54,380.54,fitbit ace 2 activity tracker kid one size,96.00,76341.6,[Bike Racks]
2,Garmin HRM-PRO Premium Heart Rate Monitor Ches...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Garmin-HRM-Pro-Real-Time...,4.7,6745,194.21,194.21,garmin hrmpro premium heart rate monitor chest...,0.00,31701.5,[]
3,Wahoo Fitness TICKR FIT Heart Rate Monitor for...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/81KoPr7Mfg...,https://www.amazon.in/Wahoo-TICKR-Monitor-iPho...,4.5,2956,119.21,165.52,wahoo fitness tickr fit heart rate monitor iph...,46.31,13302.0,[]
4,POLAR Verity Sense Optical Heart Rate Sensor,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/POLAR-Verity-Sense-Optic...,4.5,1906,212.16,287.86,polar verity sense optical heart rate sensor,75.70,8577.0,[]
5,ROTTO Cycling Jersey Mens Bike Shirt Short Sle...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/ROTTO-Cycling-Jersey-Shi...,4.4,1742,88.78,126.83,rotto cycling jersey men bike shirt short slee...,38.05,7664.8,"[Jerseys, Shorts]"
6,Lifelong Conqueror Freeride Shimano Gear Cycle...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/81VZgPyUj4...,https://www.amazon.in/Lifelong-Freeride-Suspen...,4.2,1145,125.99,233.99,lifelong conqueror freeride shimano gear cycle...,108.00,4809.0,[]
7,Geekay 24 volt 250 watt pmdc e cycle Coversion...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/71vNVnmuS1...,https://www.amazon.in/Geekay-Original-PEDAL-AS...,3.7,272,91.19,135.23,geekay 24 volt 250 watt pmdc e cycle coversion...,44.04,1006.4,[]
8,Hero Hawk Nuage 27T Single Speed Hybrid / Road...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/71oCGmZPIe...,https://www.amazon.in/Hero-Nuage-Single-Speed-...,3.5,241,96.60,144.00,hero hawk nuage 27t single speed hybrid road b...,47.40,843.5,[Road Bikes]
9,CRADIAC Discover PRO | 700 * 35 C Hybrid | Sin...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/711FIeLIx1...,https://www.amazon.in/CRADIAC-Discover-Unisex-...,4.3,157,120.32,527.99,cradiac discover pro 700 35 c hybrid single sp...,407.67,675.1,[]


## Most common purchase sub_category by customer cluster
Cluster 0
- Tires and Tubes      3410
- Bottles and Cages    1762
- Caps                  485
- Helmets               331
- Cleaners              172

Cluster 1
- Tires and Tubes      2545
- Bottles and Cages    1207
- Helmets               706
- Caps                  337
- Jerseys               277

Cluster 2
- Helmets            1313
- Road Bikes          900
- Jerseys             533
- Tires and Tubes     328
- Mountain Bikes      297

Cluster 3
- Mountain Bikes     1133
- Road Bikes          543
- Touring Bikes       388
- Jerseys             131
- Shorts              105

In [43]:
# lets make a list of categories for each cluster
clus_0_cats = ['Tires and Tubes', 'Bottles and Cages', 'Caps', 'Helmets', 'Cleaners']
clus_1_cats = ['Tires and Tubes', 'Bottles and Cages', 'Helmets', 'Caps', 'Jerseys']
clus_2_cats = ['Helmets', 'Road Bikes', 'Jerseys', 'Tires and Tubes', 'Mountain Bikes']
clus_3_cats = ['Mountain Bikes', 'Road Bikes', 'Touring Bikes', 'Jerseys', 'Shorts']

In [30]:

# group_0[(group_0.cat_matches == 'Tires and Tubes')|
#         (group_0.cat_matches == 'Bottles and Cages')|
#         (group_0.cat_matches == 'Caps')|
#         (group_0.cat_matches == 'Helmets')|
#         (group_0.cat_matches == 'Cleaners')]

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating,cat_matches


In [31]:
# group_2[(group_2.cat_matches == 'Helmets') |
#         (group_2.cat_matches == 'Road Bikes')|
#         (group_2.cat_matches == 'Jerseys')|
#         (group_2.cat_matches == 'Tires and Tubes')|
#         (pd.Series(group_2.cat_matches).isin(['Mountain Bikes']))]

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating,cat_matches


In [32]:
pd.Series('Mountain Bikes').isin(['Mountain Bikes', 'Bike Racks'])

0    True
dtype: bool

In [33]:
group_3.cat_matches[1][0]

'Bike Racks'

In [55]:
# ['Mountain Bikes', 'Road Bikes', 'Touring Bikes', 'Jerseys', 'Shorts']
# clus_matches = []
# for i, product in enumerate(group_3.cat_matches):
#     for match in product:
#         print(match)
#         if match in clus_3_cats:
#             print(i)
#             clus_matches.append(i)
# set(clus_matches)

In [54]:
# group_3[group_3.index.isin(clus_matches)]

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating,cat_matches
5,ROTTO Cycling Jersey Mens Bike Shirt Short Sle...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/ROTTO-Cycling-Jersey-Shi...,4.4,1742,88.78,126.83,rotto cycling jersey men bike shirt short slee...,38.05,7664.8,"[Jerseys, Shorts]"
8,Hero Hawk Nuage 27T Single Speed Hybrid / Road...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/71oCGmZPIe...,https://www.amazon.in/Hero-Nuage-Single-Speed-...,3.5,241,96.60,144.00,hero hawk nuage 27t single speed hybrid road b...,47.40,843.5,[Road Bikes]
13,BSA cycles Unisex-Adult Steel Photon Ex 18 Fra...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/BSA-Cycles-Photon-Bicycl...,4.1,77,107.39,119.99,bsa cycle unisexadult steel photon ex 18 frame...,12.60,315.7,[Road Bikes]
14,Hercules Flunk HT 26T Single Speed Road Cycle ...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/810lbls7yT...,https://www.amazon.in/Hercules-Flunk-HT-Single...,3.3,95,102.30,128.39,hercules flunk ht 26t single speed road cycle ...,26.09,313.5,[Road Bikes]
15,Amazon Brand - Symactive Sprinter S2000 Series...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/81HTjWJxTU...,https://www.amazon.in/Amazon-Brand-Symactive-S...,3.6,81,99.59,384.00,amazon brand symactive sprinter s2000 series 2...,284.41,291.6,[Mountain Bikes]
16,Hercules Roadeo Turner 2016 26T Men's 21 Speed...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Hercules-Roadeo-Turner-A...,3.6,67,179.99,227.99,hercules roadeo turner 2016 26t men 21 speed p...,48.00,241.2,[Road Bikes]
18,"FIFTY-FIFTY Mountain Bike Riser Handlebar, Alu...",sports & fitness,Cycling,https://m.media-amazon.com/images/I/518uEvyE+C...,https://www.amazon.in/FIFTY-FIFTY-Mountain-Han...,4.6,47,130.78,186.83,fiftyfifty mountain bike riser handlebar alumi...,56.05,216.2,[Mountain Bikes]
19,Cockatoo Men's CBC-05 Elite Series 26T & 21 Sp...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/81HtkZs7jA...,https://www.amazon.in/Cockatoo-CBC-05-Carbon-M...,3.7,57,203.99,226.80,cockatoo men cbc05 elite series 26t 21 speed c...,22.81,210.9,[Mountain Bikes]
20,Amardeep cycles Unisex Road Bike 700C with 14 ...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/717FHb50-g...,https://www.amazon.in/Amardeep-cycles-Speed-Sh...,3.9,53,191.99,300.00,amardeep cycle unisex road bike 700c 14 speed ...,108.01,206.7,[Road Bikes]
22,Geekay Hashtag 27.5t Single Speed Steel Frame ...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/81ozIYXzHv...,https://www.amazon.in/Geekay-WithoutGear-5-7in...,4.1,40,92.39,163.19,geekay hashtag 275t single speed steel frame 1...,70.80,164.0,[Mountain Bikes]


In [56]:
clus_matches = []
for i, product in enumerate(group_0.cat_matches):
    for match in product:
        if match in clus_0_cats:
            clus_matches.append(i)
clus_0_products = group_0[group_0.index.isin(clus_matches)]
            
clus_matches = []
for i, product in enumerate(group_1.cat_matches):
    for match in product:
        if match in clus_1_cats:
            clus_matches.append(i)
clus_1_products = group_1[group_1.index.isin(clus_matches)]
            
clus_matches = []
for i, product in enumerate(group_2.cat_matches):
    for match in product:
        if match in clus_2_cats:
            clus_matches.append(i)
clus_2_products = group_2[group_2.index.isin(clus_matches)]

clus_matches = []
for i, product in enumerate(group_3.cat_matches):
    for match in product:
        if match in clus_3_cats:
            clus_matches.append(i)
clus_3_products = group_3[group_3.index.isin(clus_matches)]


In [57]:
clus_0_products.head(10)

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating,cat_matches
0,Fitkit Classic Bottle Shaker 700ml,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Fitkit-Classic-Bottle-Sh...,4.1,18123,2.64,5.39,fitkit classic bottle shaker 700ml,2.75,74304.3,[Bottles and Cages]
2,Boldfit Gym Shaker for Protein Shake Leakproof...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Boldfit-Typhoon-Shaker-L...,4.1,8052,2.75,2.75,boldfit gym shaker protein shake leakproof sha...,0.00,33013.2,[Bottles and Cages]
5,SignoraWare Charger Stainless Steel Protein Sh...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Signoraware-Charger-Shak...,4.3,6596,6.53,7.79,signoraware charger stainless steel protein sh...,1.26,28362.8,[Bottles and Cages]
6,Geekey Multi-Tool – Key Shaped Pocket Tool for...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Geekey-Multi-Tool-Keycha...,4.6,6083,7.19,11.99,geekey multitool key shaped pocket tool keycha...,4.80,27981.8,[Bottles and Cages]
13,"DOVEAZ® Sports Water Bottle for Gym, Protein S...",sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Sports-Unbreakable-Freez...,4.4,2759,2.75,7.19,doveaz sport water bottle gym protein shaker b...,4.44,12139.6,[Bottles and Cages]
15,NIVIA G 20-20 Plastic Sipper/Bottle - Black (6...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Nivia-20-20-Plastic-Sipp...,3.7,2839,1.77,2.04,nivia g 2020 plastic sipper bottle black 600 m...,0.27,10504.3,[Bottles and Cages]
16,Strauss Bicycle Bottle Holder (Black),sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Strauss-ST-1347-Plastic-...,3.7,2507,2.39,4.79,strauss bicycle bottle holder black,2.40,9275.9,[Bottles and Cages]
23,Lista Adjustable Bicycle Bottle Holder Seat wi...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Lista028-Adjustable-Bicy...,3.3,1373,2.39,5.15,lista adjustable bicycle bottle holder seat qu...,2.76,4530.9,[Bottles and Cages]
25,FASTPED ® Aluminum Portable Drinking Cup Water...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/FASTPED-Aluminum-Portabl...,3.6,951,2.39,11.99,fastped aluminum portable drinking cup water b...,9.60,3423.6,"[Bike Stands, Bottles and Cages, Bike Racks]"
35,eForChina Aluminum Portable Drinking Cup Water...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/812tsjU8hv...,https://www.amazon.in/eForChina-Aluminum-Porta...,3.6,523,2.20,3.59,eforchina aluminum portable drinking cup water...,1.39,1882.8,"[Bike Stands, Bottles and Cages, Bike Racks]"


In [58]:
len(clus_0_products), len(clus_1_products), len(clus_2_products), len(clus_3_products)

(73, 12, 12, 34)

In [59]:
clus_1_products.head()

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating,cat_matches
6,Speedex Stainless Steel Vacuum Insulated Flask...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Speedex-Stainless-Thermo...,4.0,298,15.59,21.59,speedex stainless steel vacuum insulated flask...,6.00,1192.0,[Bottles and Cages]
14,KAMACHI Cycling Skating Helmet with Thick Exte...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/61pHoc+-Oe...,https://www.amazon.in/Kamachi-Professional-Cyc...,3.8,103,16.79,21.00,kamachi cycling skating helmet thick extended ...,4.21,391.4,[Helmets]
15,Lifelong Adjustable Cycling Helmet with Detach...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/71q10atG9V...,https://www.amazon.in/Lifelong-Adjustable-Deta...,4.1,78,9.59,23.99,lifelong adjustable cycling helmet detachable ...,14.40,319.8,"[Helmets, Mountain Bikes]"
26,ZAKPRO MTB Inmold Cycling Helmet with Rear LED...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/61QWn3yRGQ...,https://www.amazon.in/ZAKPRO-Inmold-Cycling-He...,4.4,20,30.71,38.39,zakpro mtb inmold cycling helmet rear led flic...,7.68,88.0,[Helmets]
42,Firefox Bicycle Helmet V-32- Size-M (Green) (5...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Firefox-Bicycle-Helmet-V...,3.9,9,26.27,26.27,firefox bicycle helmet v32 sizem green 5458 cm,0.00,35.1,[Helmets]


In [60]:
cluster = 2

In [61]:
if cluster == 0:
    clus_0_cats = ['Tires and Tubes', 'Bottles and Cages', 
                   'Caps', 'Helmets', 'Cleaners']
    group_0 = products[(products.actual_price <20)]
    group_0 = group_0.sort_values('prod_rating', ascending=False).\
            reset_index().drop(columns='index')
    clus_matches = []
    for i, product in enumerate(group_0.cat_matches):
        for match in product:
            if match in clus_0_cats:
                clus_matches.append(i)
    clus_0_products = group_0[group_0.index.isin(clus_matches)]
    return clus_0_products
    
elif cluster == 1:
    clus_1_cats = ['Tires and Tubes', 'Bottles and Cages', 
                   'Helmets', 'Caps', 'Jerseys']
    group_1 = products[(products.actual_price >= 20) & (products.actual_price < 40)]
    group_1 = group_1.sort_values('prod_rating', ascending=False).\
            reset_index().drop(columns='index')
    clus_matches = []
    for i, product in enumerate(group_1.cat_matches):
        for match in product:
            if match in clus_1_cats:
                clus_matches.append(i)
    clus_1_products = group_1[group_1.index.isin(clus_matches)]
    return clus_1_products
    
elif cluster == 2:
    clus_2_cats = ['Helmets', 'Road Bikes', 'Jerseys', 
                   'Tires and Tubes', 'Mountain Bikes']
    group_2 = products[(products.actual_price >= 40) & (products.actual_price < 100)]
    group_2 = group_2.sort_values('prod_rating', ascending=False).\
            reset_index().drop(columns='index')
    clus_matches = []
    for i, product in enumerate(group_2.cat_matches):
        for match in product:
            if match in clus_2_cats:
                clus_matches.append(i)
    clus_2_products = group_2[group_2.index.isin(clus_matches)]
    return clus_2_products
    
elif cluster == 3:
    clus_3_cats = ['Mountain Bikes', 'Road Bikes', 
                   'Touring Bikes', 'Jerseys', 'Shorts']
    group_3 = products[(products.actual_price >= 100)]
    group_3 = group_3.sort_values('prod_rating', ascending=False).\
            reset_index().drop(columns='index')
    clus_matches = []
    for i, product in enumerate(group_3.cat_matches):
        for match in product:
            if match in clus_3_cats:
                clus_matches.append(i)
    clus_3_products = group_3[group_3.index.isin(clus_matches)]
    return clus_3_products

SyntaxError: 'return' outside function (1549830986.py, line 13)

In [62]:
def get_category_list():
    sub_cat_list = ['Tires and Tubes', 'Gloves', 'Helmets', 'Bike Stands',
                   'Mountain Bikes', 'Hydration Packs', 'Jerseys', 'Fenders',
                   'Cleaners', 'Socks', 'Caps', 'Touring Bikes', 'Bottles and Cages',
                   'Vests', 'Road Bikes', 'Bike Racks', 'Shorts']
    cat_list = w.get_cat_list(sub_cat_list, extra_words=['bike', 'bikes'])
    regex_list = [r'.*tire.*|.*tube.*',
                  r'.*glove.*',
                  r'.*helmet.*',
                  r'.*stand.*',
                  r'.*mountain.*',
                  r'.*hydration.*',
                  r'.*jersey.*',
                  r'.*fender.*',
                  r'.*cleaner.*',
                  r'.*sock.*',
                  r'.*cap.*',
                  r'.*touring.*',
                  r'.*bottle.*|.*cage.*',
                  r'.*vest.*',
                  r'.*road.*',
                  r'.*rack.*',
                  r'.*short.*']
    cat_df = pd.concat([
                pd.Series(sub_cat_list), 
                pd.Series(cat_list),
                pd.Series(regex_list)
                ], axis=1).\
                rename(columns={0:'shop_cat', 
                                1:'lemmed_cat',
                                2:'regexp'})
    
    return cat_df

In [66]:
def get_products():
    return w.wrangle_products()

In [67]:
def get_cluster_product_rec(cluster):
    
    cat_df = get_category_list()
    products = get_products()
    
    cat_matches = {}
    for i, product in enumerate(products.name_preped):
        matching_cats = []
        for j, cat in enumerate(cat_df.regexp):
            if re.search(cat, product):
                matching_cats.append(cat_df.shop_cat[j])
            cat_matches[i] = matching_cats

    products = pd.concat([products, pd.Series(cat_matches)], axis=1).\
        rename(columns={0:'cat_matches'})
    
    if cluster == 0:
        clus_0_cats = ['Tires and Tubes', 'Bottles and Cages', 
                       'Caps', 'Helmets', 'Cleaners']
        group_0 = products[(products.actual_price <20)]
        group_0 = group_0.sort_values('prod_rating', ascending=False).\
                reset_index().drop(columns='index')
        clus_matches = []
        for i, product in enumerate(group_0.cat_matches):
            for match in product:
                if match in clus_0_cats:
                    clus_matches.append(i)
        clus_0_products = group_0[group_0.index.isin(clus_matches)]
        return clus_0_products

    elif cluster == 1:
        clus_1_cats = ['Tires and Tubes', 'Bottles and Cages', 
                       'Helmets', 'Caps', 'Jerseys']
        group_1 = products[(products.actual_price >= 20) & (products.actual_price < 40)]
        group_1 = group_1.sort_values('prod_rating', ascending=False).\
                reset_index().drop(columns='index')
        clus_matches = []
        for i, product in enumerate(group_1.cat_matches):
            for match in product:
                if match in clus_1_cats:
                    clus_matches.append(i)
        clus_1_products = group_1[group_1.index.isin(clus_matches)]
        return clus_1_products

    elif cluster == 2:
        clus_2_cats = ['Helmets', 'Road Bikes', 'Jerseys', 
                       'Tires and Tubes', 'Mountain Bikes']
        group_2 = products[(products.actual_price >= 40) & (products.actual_price < 100)]
        group_2 = group_2.sort_values('prod_rating', ascending=False).\
                reset_index().drop(columns='index')
        clus_matches = []
        for i, product in enumerate(group_2.cat_matches):
            for match in product:
                if match in clus_2_cats:
                    clus_matches.append(i)
        clus_2_products = group_2[group_2.index.isin(clus_matches)]
        return clus_2_products

    elif cluster == 3:
        clus_3_cats = ['Mountain Bikes', 'Road Bikes', 
                       'Touring Bikes', 'Jerseys', 'Shorts']
        group_3 = products[(products.actual_price >= 100)]
        group_3 = group_3.sort_values('prod_rating', ascending=False).\
                reset_index().drop(columns='index')
        clus_matches = []
        for i, product in enumerate(group_3.cat_matches):
            for match in product:
                if match in clus_3_cats:
                    clus_matches.append(i)
        clus_3_products = group_3[group_3.index.isin(clus_matches)]
        return clus_3_products

In [68]:
get_cluster_product_rec(0)

Opening local amazon_cycling.csv file


,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating,cat_matches
0,Fitkit Classic Bottle Shaker 700ml,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Fitkit-Classic-Bottle-Sh...,4.1,18123,2.64,5.39,fitkit classic bottle shaker 700ml,2.75,74304.3,[Bottles and Cages]
2,Boldfit Gym Shaker for Protein Shake Leakproof...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Boldfit-Typhoon-Shaker-L...,4.1,8052,2.75,2.75,boldfit gym shaker protein shake leakproof sha...,0.00,33013.2,[Bottles and Cages]
5,SignoraWare Charger Stainless Steel Protein Sh...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Signoraware-Charger-Shak...,4.3,6596,6.53,7.79,signoraware charger stainless steel protein sh...,1.26,28362.8,[Bottles and Cages]
6,Geekey Multi-Tool – Key Shaped Pocket Tool for...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Geekey-Multi-Tool-Keycha...,4.6,6083,7.19,11.99,geekey multitool key shaped pocket tool keycha...,4.80,27981.8,[Bottles and Cages]
13,"DOVEAZ® Sports Water Bottle for Gym, Protein S...",sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Sports-Unbreakable-Freez...,4.4,2759,2.75,7.19,doveaz sport water bottle gym protein shaker b...,4.44,12139.6,[Bottles and Cages]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,IndiaLot® Bicycle Inner Tube 24x 2.125/1.95/1....,sports & fitness,Cycling,https://m.media-amazon.com/images/I/21OrJDo0FL...,https://www.amazon.in/IndiaLot%C2%AE-Bicycle-T...,0.0,0,3.59,5.99,indialot bicycle inner tube 24x 2125 195 190 1...,2.40,0.0,[Tires and Tubes]
825,LIGHTER HOUSE™ Bike Handlebar Bag Waterproof B...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/LIGHTER-HOUSETM-Handleba...,0.0,0,5.99,11.70,lighter housetm bike handlebar bag waterproof ...,5.71,0.0,[Caps]
827,Vandana Enterprises CSTVI Sports Gym Shaker/Wa...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Sports-Shaker-Bottles-Pr...,0.0,0,1.91,5.99,vandana enterprise cstvi sport gym shaker wate...,4.08,0.0,[Bottles and Cages]
834,"Bike 24 Automobile,Alluminium Alloy Ring Bell ...",sports & fitness,Cycling,https://m.media-amazon.com/images/I/41O19cASwq...,https://www.amazon.in/Automobile-Alluminium-Al...,0.0,0,2.39,11.99,bike 24 automobilealluminium alloy ring bell b...,9.60,0.0,[Caps]


In [69]:
prods_0 = get_cluster_product_rec(0)

Opening local amazon_cycling.csv file
